<a href="https://colab.research.google.com/github/lpc49/LuxPollen/blob/main/LuxPollen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pollen in Luxembourg - Data loading

In [1]:
import pandas as pd

## Web scraping from Pollen.lu

In this section we scrape the data directly from the pollen.lu website. We do not use data from https://data.public.lu/en/ as it is not updated.

### Example for the first week of 1992

In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
response = requests.get("http://www.pollen.lu/index.php?qsPage=data&year=1992&week=0&qsLanguage=Fra")
response.status_code          # We expect 200 as a response status from the HTML GET request

200

In [4]:
soup = BeautifulSoup(response.text, 'html.parser')
soup.title                    # We expect the following title: <title>Pollen</title>

<title>Pollen</title>

In [5]:
html_tables = soup.find_all('table')    # Storing html tables

We can look in the tables and see that the pollen data is stored on the table number 5. 
<br>
The table does not present the header in a standard way (i.e. in a \<th> HTML tag) but rather as a sub-table inside the table's first row (first \<tr> HTML tag in the result, see below). 
<br>
We also already note that the data is split by weeks, with the url to weekly data stored in the \<option> tags. 

In [6]:
pollen_table = html_tables[5]
print(pollen_table)

<table width="100%">
<tr>
<td width="5"> </td>
<td>
<div class="content">
<h1>Données de l'année 1992</h1>
<p>La saison pollinique est terminée et on va vous renseigner dès que les premiers pollens d’arbres sont dans l’air.
(Actualisation: 01.10.2021)</p>
<form action="index.php?qsPage=data&amp;year=1992&amp;week=1&amp;qsLanguage=Fra" method="post" name="week">
<p align="center">
Faites un choix:<br>
<select name="cboWeek" onchange="jumpMenu('parent',this,0)">
<option selected="" value="index.php?qsPage=data&amp;year=1992&amp;week=0&amp;qsLanguage=Fra">
            du 1992-01-01 au 1992-01-04            </option>
<option value="index.php?qsPage=data&amp;year=1992&amp;week=1&amp;qsLanguage=Fra">
            du 1992-01-05 au 1992-01-11            </option>
<option value="index.php?qsPage=data&amp;year=1992&amp;week=2&amp;qsLanguage=Fra">
            du 1992-01-12 au 1992-01-18            </option>
<option value="index.php?qsPage=data&amp;year=1992&amp;week=3&amp;qsLanguage=Fra">
        

While we are here we extract the last actualization date from the free text of the page.

In [7]:
from datetime import datetime

In [8]:
date_start = pollen_table.text.find('Actualisation: ')+15                             # index where the date starts in the free text of the page
actualization_date_str = pollen_table.text[date_start:date_start+10]                  # actualization date as a string
# actualization_date_str = '01.10.2021'                                               # uncomment to use 1st of October 2021 by default

actualization_date = datetime.strptime(actualization_date_str, '%d.%m.%Y')            # converting to date format
actualization_date

datetime.datetime(2021, 10, 1, 0, 0)

Since we will apply the same procedure to scrape data for all relevant weeks, we define the following function:

In [9]:
def pollen_df_from_table(pollen_table):                   # takes the HTML pollen tables as input for a given week, and outputs the formatted dataframe 
    dfs = pd.read_html(str(pollen_table))                 # the HTML pollen table contains itself 2 tables: the one with pollen data with index 0, and the header with index 1
    df = dfs[0].iloc[1:, :].copy()                        # df now stores the pollen data, ignoring the first row containing free text
    df.columns = dfs[1].values.tolist()[0]                # adding the header to df
    df = df.transpose()                                   # transposing to get the species as columns and dates as row
    df.columns = df.iloc[1:2, :].values.tolist()          # defining the header as the species name, in Latin and dropping the other languages (to avoid multi-indexing)
    df = df.drop(['Français', 'Latin', 'Deutsch', 'Lëtzebuergesch'])
    df.index.name = 'Date'
    df.index = pd.to_datetime(df.index)                   # making sure the index is in date type
    df = df.astype(float)                                 # making sure the content is in float type
    return df

In [10]:
pollen_df_from_table(pollen_table)

,Ambrosia,Artemisia,Asteraceae,Alnus,Betula,Ericaceae,Carpinus,Castanea,Quercus,Chenopodium,Cupressaceae,Acer,Fraxinus,Gramineae,Fagus,Juncaceae,Aesculus,Larix,Corylus,Juglans,Umbellifereae,Ulmus,Urtica,Rumex,Populus,Pinaceae,Plantago,Platanus,Salix,Cyperaceae,Filipendula,Sambucus,Tilia
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1992-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1992-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1992-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1992-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Looping for each year and week

We start by storing all the weekly URLs in a list. 
<br> 
The week numbering is not very consistent (the first week of the year is sometimes 0 or 1, the last week is sometimes 39 or 51, there are erroneous URLs for year 2001, etc).

In [11]:
weekly_url = []

for year in range(1992, actualization_date.year +1):

    url_year = 'http://www.pollen.lu/index.php?qsPage=data&year='+str(year)+'&week=0&qsLanguage=Fra'  # this page contains the list of URLs for weekly data
    response = requests.get(url_year)
    soup = BeautifulSoup(response.text, 'html.parser')
    html_tables = soup.find_all('table')

    link_table = html_tables[5]                                                                       # in table 5, the 'option' HTML tags contain the URLs for weekly data
    for option in link_table.find_all('option'):
        link = option['value']
        url_year_week = 'http://www.pollen.lu/'+link
        weekly_url.append(url_year_week)


In [12]:
# Uncomment below to see the url list 
# weekly_url

We note that for year 2001, weeks 22 through 25, the data doesn't exist, which brings trouble later. We thus remove these URLs.

In [13]:
weekly_url.remove('http://www.pollen.lu/index.php?qsPage=data&year=2001&week=&qsLanguage=Fra')    # removing url for year 2001, week 22
weekly_url.remove('http://www.pollen.lu/index.php?qsPage=data&year=2001&week=&qsLanguage=Fra')    # removing url for year 2001, week 23
weekly_url.remove('http://www.pollen.lu/index.php?qsPage=data&year=2001&week=&qsLanguage=Fra')    # removing url for year 2001, week 24
weekly_url.remove('http://www.pollen.lu/index.php?qsPage=data&year=2001&week=&qsLanguage=Fra')    # removing url for year 2001, week 25


We now fetch the weekly data from each of the URLs. Given that there are around 1000 URLs, the process may take a few minutes, which is acceptable for our purpose.

In [14]:
pollen_dfs = []                                                 # this will be a list of small dataframes (for each week) that we concatenate at the end of the loop

for url_weekly_data in weekly_url:
        response = requests.get(url_weekly_data)
        soup = BeautifulSoup(response.text, 'html.parser')
        html_tables = soup.find_all('table')
        pollen_table = html_tables[5]                           # as above we see that the weekly pollen data is in table 5
        pollen_df = pollen_df_from_table(pollen_table)          # formatting the weekly pollen data in a dataframe using the predefined function pollen_df_from_table
        pollen_dfs.append(pollen_df)                            # adding the weekly pollen dataframe to the list 

pollen_data = pd.concat(pollen_dfs, ignore_index=False)         # concatenating all the weekly pollen dataframes into a single result dataframe

## Acquiring weather data from data.public.lu and merging

We first collect the weather data from data.public.lu

In [15]:
weather_data = pd.read_csv('https://data.public.lu/en/datasets/r/a67bd8c0-b036-4761-b161-bdab272302e5', encoding='latin', index_col=0, parse_dates=True, dayfirst=True)
weather_data.columns = ['High Temperature','Low Temperature', 'Precipitation']
weather_data.index.name = 'Date'

In [16]:
weather_data

,High Temperature,Low Temperature,Precipitation
Date,,,
1947-01-01,0.7,-0.8,0.0
1947-01-02,0.0,-1.5,0.0
1947-01-03,1.6,-2.8,0.0
1947-01-04,-1.6,-9.2,0.0
1947-01-05,-4.0,-12.4,0.0
...,...,...,...
2021-09-26,18.7,12.7,0.6
2021-09-27,18.3,10.7,1.2
2021-09-28,15.8,9.2,0.0


We now merge the pollen and weather data in a single dataframe using an outer join, and we keep only the data for relevant dates.

In [17]:
data = pd.merge(weather_data, pollen_data, left_on='Date', right_on='Date', how='outer')
data = data[(data.index >= '1992-01-01') & (data.index < actualization_date)]

We fiil the NA values with zeroes, since NA are either found for pollen data between October and December (where there is no pollen in the air), or in June 2001 as it is missing from the original website (see remarks in previous section).

In [18]:
data = data.fillna(0)                                                                  # 

We add mean temperature, year and day of year to the dataframe as it will be useful for visualization.

In [19]:

data['Mean Temperature'] = (data['High Temperature'] + data['Low Temperature'])/2

data['Year'] = data.index                                                              # Storing the Year and Day of year as it will be useful later
data['Year'] = data['Year'].dt.year

data['Day of year'] = data.index
data['Day of year'] = data['Day of year'].dt.dayofyear

data

,High Temperature,Low Temperature,Precipitation,"(Ambrosia,)","(Artemisia,)","(Asteraceae,)","(Alnus,)","(Betula,)","(Ericaceae,)","(Carpinus,)","(Castanea,)","(Quercus,)","(Chenopodium,)","(Cupressaceae,)","(Acer,)","(Fraxinus,)","(Gramineae,)","(Fagus,)","(Juncaceae,)","(Aesculus,)","(Larix,)","(Corylus,)","(Juglans,)","(Umbellifereae,)","(Ulmus,)","(Urtica,)","(Rumex,)","(Populus,)","(Pinaceae,)","(Plantago,)","(Platanus,)","(Salix,)","(Cyperaceae,)","(Filipendula,)","(Sambucus,)","(Tilia,)",Mean Temperature,Year,Day of year
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1992-01-01,-0.7,-1.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.00,1992,1
1992-01-02,-0.6,-2.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.65,1992,2
1992-01-03,2.2,-4.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.10,1992,3
1992-01-04,6.9,2.1,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.50,1992,4
1992-01-05,7.8,6.4,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.10,1992,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-26,18.7,12.7,0.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.70,2021,269
2021-09-27,18.3,10.7,1.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.50,2021,270
2021-09-28,15.8,9.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.50,2021,271


We now store the data as a csv file.

In [20]:
from google.colab import files

In [21]:
data.to_csv('data.csv', index=True)
files.download('data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>